## toBigs 팀 노트북 참고

`psutil` -> 우리는 사용하지 않을 것이므로 해당 셀 무시
- `pip install psutil` 로 설치
- 컴퓨터 리소스 확인 및 정보검색용 크로스 플랫폼 라이브러리
- 크로스 플랫폼?
    - Java: JVM이라는 가상 공간에서 컴파일 되므로 각 os에 맞는 컴파일러로 컴파일할 필요가 없음
    - 게임, 리액트네이티브, 일렉트론 등이 크로스플랫폼 지원 도구다.

### 기상청 데이터 전처리

- 추가로 수집한 데이터가 정확히 어떤 날씨 데이터인지 나오지 않아서 '출처' 확인
    - https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572
    - 단기예보 (단기예보는 예보기간과 구역을 시·공간적으로 세분화하여 발표하는 예보입니다. 지역별 시간별 차이로 인한 수요자의 불편을 최소화하기 위해 전국을 5km*5km 간격의 격자로 나누어 읍, 면, 동 단위의 행정구역 중심으로 상세한 날씨를 제공합니다.)
    - 2016년부터 있다고 하는데 왜... 2021년부터 있지? -> (구) 단기예보로 가면 확인 가능

In [1]:
# load library

# 필수
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 선택
from tqdm import tqdm   # 진행상황 표시 바, 아랍어로 progress(taqadum)
from glob import glob   # 데이터 로드 (폴더간 이동)
from datetime import datetime, timedelta    # 아마도 시간측정에 사용

# 기본
import math
import time
import random

In [2]:
# 시각화를 위한 설정
# plt.style.use("fivethirtyeight")  # (선택) css로 plt 시각화 결과 제어 / 선택지 있음 (plt)
plt.rc('font', family='NanumBarunGothic')   # (기본) 한글 깨지지 않도록 폰트설ㅈ어
plt.rc('axes', unicode_minus=False)     # (기본) 마이너스 부호 깨지지 않도록
warnings.filterwarnings(action='ignore')    # 필요없는 경고메시지 무시

In [3]:
# 경로 목록
# baseline 참고: https://dacon.io/competitions/official/235720/codeshare/2488?page=1&dtype=recent

root_path = "./data/"
files = sorted(glob(root_path+'*.csv'))

In [4]:
files

['./data/dangjin_fcst_data.csv',
 './data/dangjin_obs_data.csv',
 './data/energy.csv',
 './data/sample_submission.csv',
 './data/site_info.csv',
 './data/ulsan_fcst_data.csv',
 './data/ulsan_obs_data.csv']

In [5]:
dangjin_fcst = pd.read_csv(files[0])
dangjin_obs = pd.read_csv(files[1])
energy = pd.read_csv(files[2])
sample_submission = pd.read_csv(files[3])
site_info = pd.read_csv(files[4])
ulsan_fcst = pd.read_csv(files[5])
ulsan_obs = pd.read_csv(files[6])

### fcst data: 동네예보 분석

In [6]:
dangjin_fcst.describe()

,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
count,162208.000000,162208.000000,162208.000000,162208.000000,162208.000000,162208.000000
mean,30.675676,12.624634,76.996147,3.639262,214.147761,2.439411
std,16.714755,9.816256,17.435418,2.627592,108.020035,1.214753
min,4.000000,-14.000000,5.000000,0.000000,0.000000,1.000000
25%,16.000000,4.000000,65.000000,1.900000,115.000000,1.000000
50%,31.000000,13.000000,80.000000,2.800000,237.000000,3.000000
75%,43.000000,21.000000,90.000000,4.600000,313.000000,4.000000
max,67.000000,36.000000,100.000000,32.299999,360.000000,4.000000


In [7]:
dangjin_fcst.head()

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0


In [38]:
dangjin_fcst[['Forecast time']].drop_duplicates().sort_index()

,Forecast time
0,2018-03-01 11:00:00
20,2018-03-01 14:00:00
39,2018-03-01 17:00:00
57,2018-03-01 20:00:00
74,2018-03-01 23:00:00
...,...
162117,2021-02-28 20:00:00
162134,2021-02-28 23:00:00
162150,2021-03-01 02:00:00
162165,2021-03-01 05:00:00


기본 제공 데이터 중 -> `기상청 동네 예보` 데이터 형식 분석
- 기상청 동네 예보는 1개월 단위로 배포 되며 실시간 정보는 '공공데이터포털'의 '동네예보 조회서비스' API를 통해 확인 가능
- https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572

1. 가지고 있는 컬럼
    - forecast time: `0000-00-00 00:00:00` 형식, **예보 발표 시점**
    - forecast: **예보 시간**
        - (ex - Forecast time:2018-03-01 11:00:00, forecast:4.0 => 2018-03-01 11:00:00에 발표한 2018-03-01 15:00:00 예보
        - 예보 할 때마다 달라지기 때문? -> 처리에 유의
    - 예보 내용
        - temperature: 영하가 포함되어있음에 주의
            - 발전량보다는 소비량에 영향을 줄 데이터
            - 또는 기기 문제 (세밀한 예측에 필요)
        - humidity: 골고루 분포되어있음, 5이상 100이하의 정수
            - 마땅한 사용처가 떠오르지 않음
        - wind speed: 3분위수인 4.6 이상은 각각 아주 드문 빈도로 나타남. binding 가능
            - 종합적인 날씨정보 평가에 필요?
        - WindDirection: 0~360
        - cloud: 1, 2, 3, 4로 나뉘며 1 > 3 > 4 > 2 순으로 많음.  
            - 1: 맑음, 2: 구름 보통, 3: 구름 많음, 4: 흐림
            - [ ] 흐린 정도 (구름의 양) 가 일사량에 직접적인 영향을 미치는지 확인 (개념적)
            - [ ] 정량화된 일사량을 구할 수 없을 시 구름의 양을 데이터로 사용 할 수 있을 듯
2. 총 162208 데이터 중 누락 데이터 없음 (`.info()`)
3. 예보는 세시간마다 이루어짐

```
# insights
- forecast를 어떻게 사용 할 것인가
- 독자적인 날씨 평가 척도 (태양광 발전) 를 마련하는것도 좋을 듯
    -> 날씨 데이터를 모두 모델에 넣는것이 아니라 일사량과의 상관관계를 파악하는 것이 먼저
```


### obs data: *종관* 기상 관측 자료

In [39]:
dangjin_obs.describe()

,지점,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
count,25626.0,25589.000000,25590.000000,25590.000000,25591.000000,21656.000000
mean,129.0,12.947657,1.901407,162.146151,72.926791,5.261636
std,0.0,10.163573,1.544590,133.924277,20.379724,3.805230
min,129.0,-19.300000,0.000000,0.000000,10.000000,0.000000
25%,129.0,4.700000,0.700000,20.000000,57.000000,1.000000
50%,129.0,13.500000,1.500000,180.000000,76.000000,6.000000
75%,129.0,21.500000,2.800000,290.000000,92.000000,9.000000
max,129.0,36.800000,11.700000,360.000000,99.000000,10.000000


In [57]:
dangjin_obs.head()

,지점,지점명,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
0,129,서산,2018-03-01 00:00,3.1,3.6,340.0,96.0,NaN
1,129,서산,2018-03-01 01:00,2.8,0.7,140.0,97.0,NaN
2,129,서산,2018-03-01 02:00,2.6,3.2,320.0,95.0,NaN
3,129,서산,2018-03-01 03:00,2.0,1.9,230.0,97.0,NaN
4,129,서산,2018-03-01 04:00,2.2,2.1,180.0,97.0,NaN


In [44]:
dangjin_obs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25626 entries, 0 to 25625
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지점         25626 non-null  int64  
 1   지점명        25626 non-null  object 
 2   일시         25626 non-null  object 
 3   기온(°C)     25589 non-null  float64
 4   풍속(m/s)    25590 non-null  float64
 5   풍향(16방위)   25590 non-null  float64
 6   습도(%)      25591 non-null  float64
 7   전운량(10분위)  21656 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.6+ MB


In [52]:
# 결과값이 적은 것 순에서 많은 것 순으로 정렬
dangjin_obs[['전운량(10분위)']].value_counts(ascending=True)

전운량(10분위)
3.0           858
4.0           915
2.0           955
1.0          1052
5.0          1121
6.0          1627
7.0          1933
8.0          2033
9.0          2382
10.0         3815
0.0          4965
dtype: int64

In [56]:
# 결측치 확인
dangjin_obs.isnull().sum()

지점              0
지점명             0
일시              0
기온(°C)         37
풍속(m/s)        36
풍향(16방위)       36
습도(%)          35
전운량(10분위)    3970
dtype: int64

- 기본 제공 데이터 중 -> `당진지역 발전소 인근 기상 관측 자료` 분석
    - 지점 : 지점 코드
    - 지점명 : 관측소 지점
        - object type
    - 일시 : 관측 시간
        - 한 시간 단위로 관측
    - 기온(°C) : 기온(°C)
    - 풍속(m/s) : 풍속(m/s)
        - 눈에 띄는 이상치는 있음: binding 으로 처리 (해도 상관 없는지 한 번 확인)
    - 풍향(16방위) : 풍향(º)
    - 습도(%) : 습도(%)
    - 전운량(10분위) : 전운량(낮을 수록 구름이 적음)
        - NaN값 O, 0이 있는 것으로 보아 결측치가 0은 아님. 
        - 맑은날이면 전운량이 0일테고 그럼 값이 빌 이유가 없는데 이유를 모르겠음. 어떻게 처리할지 고민

```
# insights
- 범주형 데이터 있음: 관측지점
- 결측치가 제법 있음: 기온, 풍속, 풍향, 습도, 전운량 (많음)
- 관측시간은 한시간에 한 번
- 풍속은 특정 값 이상을 값 하나로 처리
```